In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import random
import torchvision.datasets as dsets
import torchvision.transforms as transforms #should be used to convert images to pytorch tensors

In [ ]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [ ]:
training_epoch = 10
batch_size = 100   #2^10
learning_rate = 1e-3

In [ ]:
mnist_train = dsets.MNIST(root='/content/drive/MyDrive/CSCE464/datasets/MNIST',
                          train= True,
                          transform= transforms.ToTensor(),
                          download= True)

mnist_test = dsets.MNIST(root='/content/drive/MyDrive/CSCE464/datasets/MNIST',
                          train= False,
                          transform= transforms.ToTensor(),
                          download= True)

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset= mnist_train,
                                          batch_size= batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [ ]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, 3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2),
    )

    """
    L1 image in shape = (100, 1, 28, 28) batch size 100, input channel 1, image size 28x28
    Conv -> (100, 32, 28, 28) batch size 100, output channel 32, image size 28,28
    pool -> (100, 32, 14, 14)

    L2 image in shape = (100, 32, 14, 14) batch size 100, input channel 32, image size 14x14
    Conv -> (100, 64, 14, 14) batch size 100, output channel 64, image size 14x14
    pool -> (100, 64, 7, 7)
    """

    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.fc = torch.nn.Linear(3136, 10, bias=True) #3136 from 64*7*7 in last pool
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out1 = self.layer1(x)
    out2 = self.layer2(out1)

    out3 = out2.view(out2.size(0), -1) #reduce from 2d data to 1d data
    out4 = self.fc(out3)

    return out4

In [ ]:
model = CNN().to(device)

In [ ]:
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(data_loader)

for epoch in range(training_epoch):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    pred = model(X).to(device)
    cost = loss(pred, Y).to(device)
    correct_pred = torch.argmax(pred, axis=1) == Y
    accuracy = correct_pred.float().mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print("Epoch {:2d} / {} Cost: {:.5f} Training ACC: {:.2f}%".format(epoch+1, 10, avg_cost, accuracy * 100))

Epoch  1 / 10 Cost: 0.22747 Training ACC: 99.00%
Epoch  2 / 10 Cost: 0.06156 Training ACC: 98.00%
Epoch  3 / 10 Cost: 0.04511 Training ACC: 98.00%
Epoch  4 / 10 Cost: 0.03736 Training ACC: 98.00%
Epoch  5 / 10 Cost: 0.03133 Training ACC: 99.00%
Epoch  6 / 10 Cost: 0.02570 Training ACC: 100.00%
Epoch  7 / 10 Cost: 0.02217 Training ACC: 97.00%
Epoch  8 / 10 Cost: 0.01781 Training ACC: 100.00%
Epoch  9 / 10 Cost: 0.01576 Training ACC: 98.00%
Epoch 10 / 10 Cost: 0.01315 Training ACC: 100.00%


In [ ]:
#test the model

model.eval()

with torch.no_grad():
  x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device) #1 to increase dimensionality
  y_test = mnist_test.test_labels.to(device)

  pred = model(x_test)
  correct_prediction = torch.torch.argmax(pred, axis=1) == y_test
  accuracy = correct_prediction.float().mean()
  print("Testing Accuracy {:.2f}".format(accuracy.item()*100))

Testing Accuracy 98.65


#Deep CNN Model (2+ layers)

In [ ]:
class CNN1(torch.nn.Module):
  def __init__(self):
    super(CNN1, self).__init__()

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 64, kernel_size=(5, 1), stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=(5, 6), stride=1),
    )

    """
    L1 image in shape = (100, 1, 28, 28) batch size 100, input channel 1, image size 28x28
    Conv -> (100, 64, 28, 28) batch size 100, output channel 32, image size 28,28
    pool -> (100, 64, 20, 23)

    L2 image in shape = (100, 64, 20, 23) batch size 100, input channel 32, image size 14x14
    Conv -> (100, 64, 20, 23) batch size 100, output channel 64, image size 14x14
    pool -> (100, 28, 3, 4)
    """

    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, kernel_size=(6, 5), stride=2, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=3),
    )

    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(128, 256, 3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2),
    )

    """
    L3 image in shape = (100, 64, 7, 7) batch size 100, input channel 64, image size 7x7
    Conv -> (100, 128, 7, 7) batch size 100, output channel 128, image size 7x7
    pool -> (100, 32, 14, 14)
    """

    self.fc = torch.nn.Linear(1536, 10, bias=True) #1536 from error in size
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out1 = self.layer1(x)
    out2 = self.layer2(out1)

    out3 = out2.view(out2.size(0), -1) #reduce from 2d data to 1d data
    out4 = self.fc(out3)

    return out4

In [ ]:
model = CNN1().to(device)

In [ ]:
total_batch = len(data_loader)

for epoch in range(training_epoch):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    pred = model(X).to(device)
    cost = loss(pred, Y).to(device)
    correct_pred = torch.argmax(pred, axis=1) == Y
    accuracy = correct_pred.float().mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print("Epoch {:2d} / {} Cost: {:.5f} Training ACC: {:.2f}%".format(epoch+1, 10, avg_cost, accuracy * 100))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x1536 and 512x10)